In [1]:
import torch
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
BATCH_SIZE = 64
NUM_WORKERS = 2
RANDOM_SEED = 123
NOISE_RATE = 0.1
CLASS_NUM = 7

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import logging
import time

class Logger:
    def __init__(self, mode='exp', title=''):
        """ log

        Args:
            mode (str): 'exp' or 'debug'. Defaults to 'exp', otherwise will not produce log gile.
            title (str): subdir name to store log file. Defaults to "".
        """
        # create logger
        self.logger = logging.getLogger()
        self.logger.setLevel(logging.INFO)  # setting level
        formatter_fh = logging.Formatter("[%(asctime)s] %(message)s")
        formatter_ch = logging.Formatter("%(message)s")

        # create file handler
        # setting path for logfile
        start_time = time.strftime('%y-%m-%d-%H%M', time.localtime(time.time()))
        log_path = os.path.join(os.getcwd(), 'logs', title)
        if not os.path.exists(log_path):
            os.makedirs(log_path)       
        log_name = os.path.join(log_path, start_time + '.log')
        
        if mode == 'exp': 
            fh = logging.FileHandler(log_name, mode='w')
            fh.setLevel(logging.INFO)  # setting level for outputs in logfile
            ## define format
            fh.setFormatter(formatter_fh)
            ## add handeler to the logger
            self.logger.addHandler(fh)

        # console handeler
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)
        ch.setFormatter(formatter_ch)
        self.logger.addHandler(ch)

In [4]:
# log file
model_id = '1'
exp_id = '1-1'
model_name = exp_id + '-' + model_id
log = Logger(mode='exp', title=exp_id)
log.logger.info("{}".format(model_name))

1-1-1


**Data**

In [5]:
from torchvision import transforms
from torch.utils import data
from PIL import Image
from numpy.testing import assert_array_almost_equal
import pandas as pd


class NoisyISIC2018(data.Dataset):
    def __init__(self, ann_file: str, img_dir: str, transform=None, target_transform=None,
                 noise_type: str = 'symmetric', noise_rate: float = 0.1, random_state: int = 123):
        """ ISIC 2018 Dataset with noisy labels

        Args:
            ann_file (str): csv annotation file path
            img_dir (str): directory path of images
            transform: input transformation
            target_transform: target transformation
            noise_type (str): noise type ('symmetric', 'asymmetric'). Defaults to 'symmetric'.
            noise_rate (float): rate of noise. Defaults to 0.1.
            random_state (int): random seed. Defaults to 123.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.noise_type = noise_type
        self.noise_rate = noise_rate
        self.img_ids, self.clean_labels = self._csv_reader(ann_file)
        self.class_num = len(self.categories)
        self.noisy_labels, self.actual_noise_rate = noisify(
            self.clean_labels, self.noise_type, self.noise_rate, self.class_num, random_state)
        print("Actual noise rate: {:.4f}".format(self.actual_noise_rate))

    def _csv_reader(self, csv_file):
        df = pd.read_csv(csv_file, header=0)
        self.categories = list(df.columns)[1:]
        self.class_dict = {}
        self.label_dict = {}
        for i, name in enumerate(self.categories):
            self.class_dict[name] = i
            self.label_dict[i] = name
        df['label'] = df.select_dtypes(['number']).idxmax(axis=1)
        df['label'] = df['label'].apply(lambda x: self.class_dict[x])
        img_ids = list(df['image'])
        labels = np.array(list(df['label']), dtype=np.int64)
        return img_ids, labels

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index

        Returns:
            tuple: (image, clean_label, noisy_label)
        """

        pth_img = os.path.join(self.img_dir, self.img_ids[idx] + '.jpg')
        img = Image.open(pth_img)
        clean_label = self.clean_labels[idx]
        noisy_label = self.noisy_labels[idx]

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            clean_label = self.target_transform(clean_label)
            noisy_label = self.target_transform(noisy_label)

        return img, clean_label, noisy_label

    def __len__(self):
        return len(self.img_ids)

    def to_names(self, nums):
        """ convert a goup of indices to string names 
        
        Args:
            nums(torch.Tensor): a list of number labels

        Return:
            a list of dermatological names
        
        """
        names = [self.label_dict[int(num)] for num in nums]
        return names


def multiclass_noisify(y, P, random_state=123):
    """ Flip classes according to transition probability matrix T.
    It expects a number between 0 and the number of classes - 1.

    Args:
        y (list): a list of index label
        P (matrix): n x n transition matrix with values between [0, 1]
        random_state (int): random seed. Defaults to 123.

    Returns:
        noisy y
    """
    assert P.shape[0] == P.shape[1]
    assert np.max(y) < P.shape[0]

    # row stochastic matrix
    assert_array_almost_equal(P.sum(axis=1), np.ones(P.shape[1]))
    assert (P >= 0.0).all()

    m = y.shape[0]
    new_y = y.copy()
    flipper = np.random.RandomState(random_state)

    for idx in np.arange(m):
        i = y[idx]
        flipped = flipper.multinomial(1, P[i, :], 1)[0]
        new_y[idx] = np.where(flipped == 1)[0]

    return new_y


def noisify_symmetric(y, noise_rate, random_state=123, nb_classes=7):
    """ noisify labels in the symmetric way
    """
    # create transition matrix
    P = np.ones((nb_classes, nb_classes))
    ## convert to other classes with equal probabilities (p = noise/(n-1))
    P = (noise_rate / (nb_classes - 1)) * P

    if noise_rate > 0.0:
        for i in range(nb_classes):
            P[i, i] = 1. - noise_rate

        noisy_y = multiclass_noisify(y, P=P, random_state=random_state)
        actual_noise_rate = (noisy_y != y).mean()

    return noisy_y, actual_noise_rate


def noisify_asymmetric(y, noise_rate, random_state=123):
    r""" noisify labels in an asymmetric way: 𝑁𝑉 <-> 𝑀𝐸𝐿, 𝐵𝐶𝐶 <-> 𝐵𝐾𝐿, 𝑉𝐴𝑆𝐶 <-> 𝐷𝐹,
        {'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
    """
    P = np.eye(7)
    n = noise_rate

    if n > 0.0:
        # 0 <-> 1
        P[0, 0], P[0, 1] = 1. - n, n
        P[1, 1], P[1, 0] = 1. - n, n

        # 2 <-> 4
        P[2, 2], P[2, 4] = 1. - n, n
        P[4, 4], P[4, 2] = 1. - n, n

        # 5 <-> 6
        P[5, 5], P[5, 6] = 1. - n, n
        P[6, 6], P[6, 5] = 1. - n, n

        # 3 <-> 6
        P[3, 3], P[3, 6] = 1. - n, n

        noisy_y = multiclass_noisify(y, P=P, random_state=random_state)
        actual_noise_rate = (noisy_y != y).mean()

    return noisy_y, actual_noise_rate


def noisify(labels, noise_type='symmetric', noise_rate=0.1, class_num=7, random_state=123):
    assert noise_rate >= 0 and noise_rate <= 1, "Noise rate is not in [0, 1]"
    if noise_rate == 0:
        return labels, 0
    if noise_type == 'symmetric':
        noisy_labels, actual_noise_rate = noisify_symmetric(
            labels, noise_rate, random_state=random_state, nb_classes=class_num)
    elif noise_type == 'asymmetric':
        noisy_labels, actual_noise_rate = noisify_asymmetric(
            labels, noise_rate, random_state=random_state)
    else:
        raise ValueError('Not Implemented')
    return noisy_labels, actual_noise_rate

In [6]:
# transforms

trans_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.4, 1), ratio=(3/4, 4/3)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

trans_test = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [7]:
# dataset

train_data = NoisyISIC2018(ann_file='../input/isic-groundtruth-for-classification/Train_GroundTruth.csv',
                           img_dir='../input/isic2018/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input',
                           transform=trans_train, noise_type='asymmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)
test_data = NoisyISIC2018(ann_file='../input/isic-groundtruth-for-classification/Test_GroundTruth.csv',
                          img_dir='../input/isic2018/ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input',
                          transform=trans_test, noise_type='symmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)
valid_data = NoisyISIC2018(ann_file='../input/isic-groundtruth-for-classification/ISIC2018_Task3_Validation_GroundTruth.csv',
                           img_dir='../input/isic2018task3validation/ISIC2018_Task3_Validation_Input',
                           transform=trans_test, noise_type='symmetric', noise_rate=NOISE_RATE, random_state=RANDOM_SEED)

Actual noise rate: 0.0940
Actual noise rate: 0.0978
Actual noise rate: 0.1295


In [8]:
# dataloader

train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=NUM_WORKERS)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
valid_loader = data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [9]:
import torch.nn as nn
from torchvision import models
from collections import OrderedDict

In [10]:
from torchvision import models
from collections import OrderedDict
import torch.nn as nn
import torch.nn.functional as F


pretrained_m = torch.load('../input/dense2015/dense201-5.pkl')
state_dict = pretrained_m.state_dict()

model = models.densenet201(pretrained=True)
classifier = nn.Sequential(OrderedDict([
    ('fc0', nn.Linear(1920, 256)),
    #('norm0', nn.BatchNorm1d(256)),
    ('dropout0', nn.Dropout(p=0.5)),
    ('relu0', nn.ReLU(inplace=True)),
    ('fc1', nn.Linear(256, 7))
]))
model.classifier = classifier
model_dict = model.state_dict()

remove_keys = []
for k,v in state_dict.items():
    if "classifier.norm0" in k:
        remove_keys.append(k)
for k in remove_keys:
        state_dict.__delitem__(k)
    
# model.features.load_state_dict(state_dict)
model.load_state_dict(state_dict)
model.to(device) 
del pretrained_m
del state_dict


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class WeightedCELoss(nn.Module):
    def __init__(self, sample_weight: torch.Tensor, num_classes=7, reduction='mean'):
        super(WeightedCELoss, self).__init__()
        self.num_classes = num_classes
        self.reduction = reduction
        self.sample_weight = sample_weight

    def forward(self, pred: torch.Tensor, labels: torch.Tensor, c: int):
        """w * y * log(p)

        Args:
            pred (torch.Tensor): 网络输出值
            labels (torch.Tensor): 标签
            sample_weight (torch.Tensor): 逐样本权重
            c:  batchIdx

        Shape:
            pred: N, C
            label: N
            sample_weight: N

        Returns:
            tensor.float: 逐样本赋权的损失值
        """

        pred = F.log_softmax(pred, dim=1)                                           # N, C
        label_one_hot = F.one_hot(labels, self.num_classes).float().to(pred.device) # C -> N, C
        loss = label_one_hot * pred                                                 # N, C
        # 赋权
        loss = - 1 * torch.sum(label_one_hot * pred, dim=1) * self.sample_weight[BATCH_SIZE*c:BATCH_SIZE*(c+1)]          # N
        
        # 规约
        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        else:
            raise ValueError('Unsupported reduction type.')

        return loss

In [12]:
import numpy as np
def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

            
def get_probs(model,data_loader):
    model.eval()
    label = []
    prob = [[1 for _ in range(7)]]
    soft = nn.Softmax(dim=-1)
    
    for x, clean_y,noise_y in train_loader:
        # setting GPU
        x = x.to(device)
        enable_dropout(model)
        z = model(x)
        p = soft(z)
        prob = np.concatenate((prob, p.detach().to('cpu')), axis=-2)
        #print(prob.shape)
    prob = prob[1::]
    return prob

def Uncertainty_Estimation(T=5):
    prob = get_probs(model, train_loader)
    probs = [prob]
    for _ in range(T-1):
        prob = get_probs(model,train_loader)
        probs.append(prob)
    prob = np.mean(probs,axis=0)
    UoSL = - 1.0 * np.sum(prob * np.log(prob + 1e-16), axis=-1)
    
    np.savetxt(exp_id+"UoSL.txt",UoSL)
    #print("prob:{:.2f}\tUoSL: {}".format(prob,UoSL))
    return prob, UoSL



In [13]:
#prob, UoSL = Uncertainty_Estimation(5)

In [14]:
UoSL = np.loadtxt("../input/11uosl/1-1UoSL.txt")
np.set_printoptions(threshold=np.inf)
print(type(UoSL))
print(type(UoSL[0]))
print(UoSL.shape)
np.savetxt(exp_id+"UoSL.txt",UoSL)


<class 'numpy.ndarray'>
<class 'numpy.float64'>
(7511,)


In [15]:
import numpy as np
import torch
import torch.nn as nn 
def norm(UoSL):
    minV = UoSL.min(0)
    maxV = UoSL.max(0)
    return (UoSL-minV)/(maxV-minV)

#UoSL = np.loadtxt("1-1UoSL.txt")
nUoSL = norm(UoSL)
tUoSL = np.mean(nUoSL)
sample_weight = np.zeros(np.shape(UoSL))
for i in range(len(nUoSL)):
    if(nUoSL[i] > tUoSL):
        sample_weight[i] = 1-nUoSL[i]
    else:
        sample_weight[i] = 1
# left = np.zeros(64-23)
# sample_weight = np.concatenate((sample_weight,left))
sample_weight= torch.from_numpy(sample_weight)


In [16]:
import torch.nn as nn
sample_weight = sample_weight.to(device)
#criterion = nn.CrossEntropyLoss()
criterion = WeightedCELoss(sample_weight = sample_weight)

In [17]:
import torch.optim as optim
import gc

class Trainer(object):
    def __init__(self, device, log, model_name: str, optimizer=None, scheduler=None, grad_bound: float = 5., start_epoch: int = 0, best_score=0, checkpoint_model=None):
        """ trainer for segmentation tasks

        Args:
            device (torch.device)
            log (Logger): logfile
            model_name (str): name of the model
            optimizer (torch.nn.optim)
            scheduler (torch.nn.optim)
            grad_bound (float): max norm of the gradients
            start_epoch (int): initial epoch
            best_score (float): metric score for early stopping
            checkpoint_model (None or nn.Module): None - train from scratch; nn.Module - reload from checkpoint
        """
        self.device = device
        self.log = log
        self.model_name = model_name
        self.grad_bound = grad_bound
        if not os.path.exists('model'):
            os.makedirs('model')
        if not os.path.exists('checkpoint'):
            os.makedirs('checkpoint')
        # path to store checkpoint
        self.pth_check = os.path.join(
            'checkpoint', 'check_' + model_name + '.pth')

        if checkpoint_model == None:
            self.epoch = start_epoch
            self.optimizer = optimizer
            self.scheduler = scheduler
            self.train_costs = []
            self.train_accs = []
            self.train_actual_accs = []
            self.val_costs = []
            self.val_accs = []
            self.val_actual_accs = []
            self.best_score = best_score
            self.patience = 0
        else:
            checkpoint = torch.load(self.pth_check)
            self.epoch = checkpoint['epoch'] + 1
            self.optimizer = checkpoint['optimizer']
            self.scheduler = checkpoint['scheduler']
            self.train_costs = checkpoint['train_costs']
            self.train_accs = checkpoint['train_accs']
            self.train_actual_accs = checkpoint['train_actual_accs']
            self.val_costs = checkpoint['val_costs']
            self.val_accs = checkpoint['val_accs']
            self.val_actual_accs = checkpoint['val_actual_accs']
            self.best_score = checkpoint['best_score']
            self.patience = checkpoint['patience']
            checkpoint_model.load_state_dict(checkpoint['model_state_dict'])

    def fit(self, model, train_loader, val_loader, criterion, max_epoch, test_period=5, early_threshold=10):
        size_train = len(train_loader)
        num_train = len(train_loader.dataset)
        size_val = len(val_loader)
        num_val = len(val_loader.dataset)
        model.train()

        for self.epoch in range(self.epoch, max_epoch):
            cost = 0
            acc = 0
            actual_acc = 0

            c = 0
            for x, clean_y, noisy_y in train_loader:
                x, clean_y, noisy_y = x.to(self.device), clean_y.to(
                    self.device), noisy_y.to(self.device)
                self.optimizer.zero_grad()
                z = model(x)
                loss = criterion(z, noisy_y, c)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), self.grad_bound) # 梯度裁剪
                self.optimizer.step()

                cost += loss.item()
                _, yhat = torch.max(z.data, 1)
                acc += (yhat == noisy_y).sum().item()
                actual_acc += (yhat == clean_y).sum().item()
                c = c+1

            self.train_costs.append(cost/size_train)
            self.train_accs.append(acc/num_train)
            self.train_actual_accs.append(actual_acc/num_train)
            self.scheduler.step()

            gc.collect()

            if self.epoch % test_period == 0:
                model.eval()
                cost, acc, noisy_acc = 0, 0, 0
                with torch.no_grad():
                    for x, clean_y, noisy_y in val_loader:
                        x, clean_y, noisy_y = x.to(self.device), clean_y.to(
                            self.device), noisy_y.to(self.device)
                        z = model(x)
                        _, yhat = torch.max(z.data, 1)
                        acc += (yhat == clean_y).sum().item()
                        noisy_acc += (yhat == noisy_y).sum().item()
                self.val_costs.append(cost/size_val)
                self.val_accs.append(noisy_acc/num_val)
                self.val_actual_accs.append(acc/num_val)

                if self.val_actual_accs[-1] >= self.best_score:
                    self.best_score = self.val_actual_accs[-1]
                    self.patience = 0
                    save_state_dict(
                        model, name="{}_dict.pth".format(self.model_name))
                else:
                    self.patience += 1
                    if self.patience >= early_threshold:
                        break

                self.log.logger.info("Epoch:{:3d} train_cost: {:.4f}\ttrain_acc: {:.4f}\tta_acc: {:.4f}\tval_cost: {:.4f}\tval_acc: {:.4f}\tva_acc: {:.4f}".format(
                    self.epoch+1, self.train_costs[-1], self.train_accs[-1], self.train_actual_accs[-1], self.val_costs[-1], self.val_accs[-1], self.val_actual_accs[-1]))

                model.train()

            self.checkpoint(self.pth_check, model)

        save_model(model, name=self.model_name+'.pkl')
        history = self.get_history()
        self.log.logger.info("Model has been saved at {}\n{}".format(
            self.model_name+'.pkl', history))
        return history


    def checkpoint(self, check_file, model):
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer': self.optimizer,
            'scheduler': self.scheduler,
            'epoch': self.epoch,
            'train_costs': self.train_costs,
            'train_accs': self.train_accs,
            'train_actual_accs': self.train_actual_accs,
            'val_costs': self.val_costs,
            'val_accs': self.val_accs,
            'val_actual_accs': self.val_actual_accs,
            'best_score': self.best_score,
            'patience': self.patience
        }
        torch.save(checkpoint, check_file)

    def get_history(self):
        history = {
            'train_costs': self.train_costs,
            'train_accs': self.train_accs,
            'train_actual_accs': self.train_actual_accs,
            'val_costs': self.val_costs,
            'val_accs': self.val_accs,
            'val_actual_accs': self.val_actual_accs,
            'best_score': self.best_score
        }
        return history


def load_model(device, name='model.pkl'):
    """
    load model from ./model/name 加载网络
    """
    pth_model = os.path.join('model', name)
    assert os.path.exists(pth_model), "Model file doesn't exist!"
    model = torch.load(pth_model, map_location=device)
    print('Load {} on {} successfully.'.format(name, device))
    return model


def save_model(model, name='model.pkl'):
    """ 
    save model to ./model/name 保存网络
    """

    if not os.path.exists('model'):
      os.makedirs('model')

    pth_model = os.path.join('model', name)
    torch.save(model, pth_model)
    print('Model has been saved to {}'.format(pth_model))


def save_state_dict(model, name='state_dict.pth'):
    """ 
    save state dict to ./model/temp/name 保存网络参数
    """

    model_dir = os.path.join('model', 'temp')
    if not os.path.exists(model_dir):
      os.makedirs(model_dir)

    pth_dict = os.path.join(model_dir, name)
    torch.save(model.state_dict(), pth_dict)
    print('state dict has been saved to {}'.format(pth_dict))


def load_state_dict(model, device, name='state_dict.pth'):
    """ 
    load model parmas from state_dict 加载网络参数
    """
    pth_dict = os.path.join('model', 'temp', name)
    assert os.path.exists(pth_dict), "State dict file doesn't exist!"
    model.load_state_dict(torch.load(pth_dict, map_location=device))
    return model

In [18]:
## hyper-params
init_lr = 1e-3
weight_decay = 1e-4
max_epoch = 100
test_period = 1
early_threshold = 40

optimizer = optim.AdamW(model.classifier.parameters(), lr=init_lr, betas=(0.9, 0.999), weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=0)

trainer = Trainer(device, log, model_name, optimizer, scheduler, checkpoint_model=None)


In [19]:
history = trainer.fit(model, train_loader, valid_loader, criterion, max_epoch, test_period, early_threshold)

Epoch:  1 train_cost: 0.6305	train_acc: 0.6885	ta_acc: 0.7506	val_cost: 0.0000	val_acc: 0.6684	va_acc: 0.7513


state dict has been saved to model/temp/1-1-1_dict.pth


Epoch:  2 train_cost: 0.5288	train_acc: 0.7079	ta_acc: 0.7663	val_cost: 0.0000	val_acc: 0.7150	va_acc: 0.8187


state dict has been saved to model/temp/1-1-1_dict.pth


Epoch:  3 train_cost: 0.5225	train_acc: 0.7055	ta_acc: 0.7702	val_cost: 0.0000	val_acc: 0.7047	va_acc: 0.7927
Epoch:  4 train_cost: 0.5009	train_acc: 0.7204	ta_acc: 0.7850	val_cost: 0.0000	val_acc: 0.7047	va_acc: 0.7927
Epoch:  5 train_cost: 0.4938	train_acc: 0.7108	ta_acc: 0.7754	val_cost: 0.0000	val_acc: 0.6995	va_acc: 0.7927
Epoch:  6 train_cost: 0.5034	train_acc: 0.7171	ta_acc: 0.7811	val_cost: 0.0000	val_acc: 0.7306	va_acc: 0.8394


...

state dict has been saved to model/temp/1-1-1_dict.pth


Epoch: 24 train_cost: 0.4213	train_acc: 0.7619	ta_acc: 0.8237	val_cost: 0.0000	val_acc: 0.7098	va_acc: 0.8083
Epoch: 25 train_cost: 0.4380	train_acc: 0.7558	ta_acc: 0.8191	val_cost: 0.0000	val_acc: 0.7047	va_acc: 0.7927
Epoch: 26 train_cost: 0.4230	train_acc: 0.7556	ta_acc: 0.8183	val_cost: 0.0000	val_acc: 0.7306	va_acc: 0.8290
Epoch: 27 train_cost: 0.4095	train_acc: 0.7625	ta_acc: 0.8212	val_cost: 0.0000	val_acc: 0.7358	va_acc: 0.8290
Epoch: 28 train_cost: 0.4104	train_acc: 0.7675	ta_acc: 0.8306	val_cost: 0.0000	val_acc: 0.7358	va_acc: 0.8290
Epoch: 29 train_cost: 0.4152	train_acc: 0.7609	ta_acc: 0.8233	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8187
Epoch: 30 train_cost: 0.4139	train_acc: 0.7657	ta_acc: 0.8269	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8135
Epoch: 31 train_cost: 0.4080	train_acc: 0.7685	ta_acc: 0.8267	val_cost: 0.0000	val_acc: 0.7358	va_acc: 0.8290
Epoch: 32 train_cost: 0.4128	train_acc: 0.7623	ta_acc: 0.8259	val_cost: 0.0000	val_acc: 0.7150	va_acc: 0.8031
Epoch: 33 

state dict has been saved to model/temp/1-1-1_dict.pth


Epoch: 41 train_cost: 0.3837	train_acc: 0.7761	ta_acc: 0.8352	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8187
Epoch: 42 train_cost: 0.3914	train_acc: 0.7787	ta_acc: 0.8420	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8290
Epoch: 43 train_cost: 0.3865	train_acc: 0.7747	ta_acc: 0.8360	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8187
Epoch: 44 train_cost: 0.3884	train_acc: 0.7759	ta_acc: 0.8384	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8187
Epoch: 45 train_cost: 0.3855	train_acc: 0.7815	ta_acc: 0.8436	val_cost: 0.0000	val_acc: 0.7254	va_acc: 0.8290
Epoch: 46 train_cost: 0.3778	train_acc: 0.7850	ta_acc: 0.8457	val_cost: 0.0000	val_acc: 0.7254	va_acc: 0.8238
Epoch: 47 train_cost: 0.3792	train_acc: 0.7794	ta_acc: 0.8373	val_cost: 0.0000	val_acc: 0.7202	va_acc: 0.8187
Epoch: 48 train_cost: 0.3718	train_acc: 0.7837	ta_acc: 0.8452	val_cost: 0.0000	val_acc: 0.7254	va_acc: 0.8238
Epoch: 49 train_cost: 0.3772	train_acc: 0.7855	ta_acc: 0.8445	val_cost: 0.0000	val_acc: 0.6995	va_acc: 0.7979
Epoch: 50 

Model has been saved to model/1-1-1.pkl


In [20]:
from sklearn import metrics
from sklearn.preprocessing import label_binarize


def evaluation(model, data_loader, categories=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']):
    y, prob, label, pred = predict(model, data_loader)
    report = metrics.classification_report(label, pred, target_names=categories, digits=3)
    roc_auc = auc_scores(y, prob)
    print("Evaluation Report:\n{}".format(report))
    print("AUC:\n{}".format(roc_auc))


@torch.no_grad()
def predict(model, data_loader):
    """ get predicted probabilities

    Returns:
        y: one-hot labels
        prob: predicted probabilities
    """
    model.eval()
    device = next(model.parameters()).device
    label = []
    prob = [[1 for _ in range(7)]]
    soft = nn.Softmax(dim=-1)

    for x, y, _ in data_loader:
        x, y = x.to(device), y.to(device)
        z = model(x)
        p = soft(z)
        prob = np.concatenate((prob, p.to('cpu')), axis=-2)
        label = np.concatenate((label, y.to('cpu')), axis=-1)
    
    prob = prob[1::]
    class_num = prob.shape[1]
    y = label_binarize(label, classes=[i for i in range(class_num)])
    pred = np.argmax(prob, axis=1)

    return y, prob, label, pred



def auc_scores(y, prob):
    class_num = prob.shape[1]

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Compute ROC curve and ROC area for each class
    for i in range(class_num):
        fpr[i], tpr[i], _ = metrics.roc_curve(y[:, i], prob[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area (computed globally)
    fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y.ravel(), prob.ravel())
    roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area (simply average on each label)
    # aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(class_num)]))
    # interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(class_num):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    # average it and compute AUC
    mean_tpr /= class_num

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = metrics.auc(fpr["macro"], tpr["macro"])

    return roc_auc

In [21]:
evaluation(model, test_loader)

Evaluation Report:
              precision    recall  f1-score   support

         MEL      0.749     0.558     0.640       283
          NV      0.895     0.963     0.928      1671
         BCC      0.802     0.761     0.781       117
       AKIEC      0.889     0.771     0.826        83
         BKL      0.792     0.713     0.750       289
          DF      1.000     0.630     0.773        27
        VASC      0.912     0.912     0.912        34

    accuracy                          0.869      2504
   macro avg      0.863     0.758     0.801      2504
weighted avg      0.863     0.869     0.863      2504

AUC:
{0: 0.9352391164964052, 1: 0.9613015762858106, 2: 0.9839801775285646, 3: 0.9937295651005509, 4: 0.9687925203277434, 5: 0.9943480016148567, 6: 0.9993331745653726, 'micro': 0.9870453281139953, 'macro': 0.9768358651523769}
